# LAB_2


In [3]:
# ============================================================================
# LABORATORIJSKA VJEŽBA 2: Newton-Raphsonov algoritam
# Predmet: Optimizacija resursa
# Univerzitet u Sarajevu - Elektrotehnički fakultet
# ============================================================================

"""
# 📚 Uvod

Ova laboratorijska vježba demonstrira implementaciju i primjenu
**Newton-Raphsonovog algoritma** za traženje lokalnih ekstrema funkcija.

## Teorijska osnova

Newton-Raphsonov algoritam koristi rekurzivnu relaciju:

$$x_{k+1} = x_k - \\frac{f'(x_k)}{f''(x_k)}$$

### Uslov zaustavljanja:
- Maksimalan broj iteracija N, ili
- $|f(x_{k+1}) - f(x_k)| < \\varepsilon$
"""

# ============================================================================
# INSTALACIJA I IMPORTOVANJE BIBLIOTEKA
# ============================================================================

# Instaliramo plotly ako nije instaliran
try:
    import plotly.graph_objects as go
except ImportError:
    print("Instaliram plotly...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'plotly', '--break-system-packages'])
    import plotly.graph_objects as go

import numpy as np
import math as m
import random
from typing import Callable, Tuple, List

print("✅ Sve biblioteke uspješno učitane!")

# ============================================================================
# ZADATAK 1: IMPLEMENTACIJA NEWTON-RAPHSON ALGORITMA
# ============================================================================

"""
# 🔧 Zadatak 1: Implementacija Newton-Raphson algoritma

Implementiramo funkciju `NR` koja prima:
- `f`: kriterijska funkcija
- `df`: prvi izvod funkcije
- `ddf`: drugi izvod funkcije
- `x0`: početna tačka pretraživanja
- `N`: maksimalan broj iteracija
- `eps`: minimalna promjena po vrijednosti funkcije
"""

def NR(f: Callable, df: Callable, ddf: Callable,
       x0: float, N: int, eps: float) -> Tuple[float, List[float], int]:
    """
    Newton-Raphson algoritam za traženje lokalnog ekstrema funkcije.
    """
    x_current = x0
    history = [x0]

    for k in range(N):
        ddf_val = ddf(x_current)

        # AKO JE DRUGI IZVOD BLIZU NULE - POMJERIMO SE MALO!
        if abs(ddf_val) < 1e-12:
            print(f"⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji {k}!")
            print(f"   Pomjeram se za random offset...")

            # Random pomjeranje za ±0.01
            offset = random.choice([-0.01, 0.01])  # ili random.uniform(-0.01, 0.01)
            x_current = x_current + offset
            history.append(x_current)

            # Nastavi sa sljedećom iteracijom
            continue

        # Newton-Raphson formula
        x_next = x_current - df(x_current) / ddf_val
        history.append(x_next)

        # Provjeravamo uslov zaustavljanja
        if abs(f(x_next) - f(x_current)) < eps:
            print(f"✓ Konvergencija postignuta nakon {k+1} iteracija")
            print(f"  Početna tačka: x0 = {x0:.6f}")
            print(f"  Finalna tačka: x* = {x_next:.6f}")
            print(f"  Vrijednost funkcije: f(x*) = {f(x_next):.6f}")
            return x_next, history, k + 1

        x_current = x_next

    print(f"⚠️ Dostignut maksimalan broj iteracija ({N})")
    print(f"  Početna tačka: x0 = {x0:.6f}")
    print(f"  Finalna tačka: x* = {x_current:.6f}")
    print(f"  Vrijednost funkcije: f(x*) = {f(x_current):.6f}")
    return x_current, history, N


# ============================================================================
# ZADATAK 2: DEFINICIJE TEST FUNKCIJA I NJIHOVIH IZVODA
# ============================================================================

"""
# 📊 Zadatak 2: Test funkcije

Definirat ćemo sve test funkcije i njihove izvode:

1. $f_1(x) = 3x^2 - 1$
2. $f_2(x) = -(16x^2 - 24x + 5)e^{-x}$
3. $f_3(x) = \\sin(x) + \\sin(\\frac{10x}{3})$
4. $f_4(x) = e^{-x}$
"""

# Funkcija 1: f(x) = 3x^2 - 1
def f1(x):
    return 3*x**2 - 1

def df1(x):
    return 6*x

def ddf1(x):
    return 6

# Funkcija 2: f(x) = -(16x^2 - 24x + 5)e^(-x)
def f2(x):
    return -(16*x**2 - 24*x + 5) * np.exp(-x)

def df2(x):
    # f'(x) = -[(32x - 24)e^(-x) - (16x^2 - 24x + 5)e^(-x)]
    # f'(x) = -e^(-x)[32x - 24 - 16x^2 + 24x - 5]
    # f'(x) = -e^(-x)[-16x^2 + 56x - 29]
    return -np.exp(-x) * (-16*x**2 + 56*x - 29)

def ddf2(x):
    # f''(x) = -e^(-x)[(-32x + 56) - (-16x^2 + 56x - 29)]
    # f''(x) = -e^(-x)[-32x + 56 + 16x^2 - 56x + 29]
    # f''(x) = -e^(-x)[16x^2 - 88x + 85]
    return -np.exp(-x) * (16*x**2 - 88*x + 85)

# Funkcija 3: f(x) = sin(x) + sin(10x/3)
def f3(x):
    return np.sin(x) + np.sin(10*x/3)

def df3(x):
    return np.cos(x) + (10/3)*np.cos(10*x/3)

def ddf3(x):
    return -np.sin(x) - (100/9)*np.sin(10*x/3)

# Funkcija 4: f(x) = e^(-x)
def f4(x):
    return np.exp(-x)

def df4(x):
    return -np.exp(-x)

def ddf4(x):
    return np.exp(-x)

print("✅ Sve funkcije i njihovi izvodi definirani!")


# ============================================================================
# FUNKCIJA ZA VIZUALIZACIJU
# ============================================================================

"""
# 📈 Funkcija za vizualizaciju rezultata

Kreiraćemo pomocnu funkciju koja će crtati:
- Grafik funkcije (crna linija)
- Lokalne ekstreme (crveni kružići ◦)
- Pronađene tačke Newton-Raphson algoritmom (zeleni križići ×)
"""

def plot_function_with_results(f, x_range, nr_points, local_extrema,
                                title, x0_values):
    """
    Crta funkciju sa označenim lokalnim ekstremima i NR rezultatima.

    Parametri:
    ----------
    f : funkcija
        Funkcija koju crtamo
    x_range : tuple
        (x_min, x_max) opseg za crtanje
    nr_points : list
        Lista tačaka pronađenih NR algoritmom
    local_extrema : list
        Lista lokalnih ekstrema (x koordinate)
    title : str
        Naslov grafika
    x0_values : list
        Lista početnih tačaka
    """
    # Generišemo x vrijednosti za crtanje
    x = np.linspace(x_range[0], x_range[1], 1000)

    # Računamo y vrijednosti
    # Pazimo na funkcije koje mogu imati probleme (npr. dijeljenje sa nulom)
    try:
        y = f(x)
    except:
        # Ako ima problema, računamo tačku po tačku
        y = np.array([f(xi) for xi in x])

    # Kreiramo figuru
    fig = go.Figure()

    # Dodajemo glavnu liniju funkcije
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        name='f(x)',
        line=dict(color='black', width=2)
    ))

    # Dodajemo lokalne ekstreme (crveni kružići)
    if local_extrema:
        x_ext = np.array(local_extrema)
        y_ext = np.array([f(xi) for xi in x_ext])
        fig.add_trace(go.Scatter(
            x=x_ext, y=y_ext,
            mode='markers',
            name='Lokalni ekstremi',
            marker=dict(color='red', size=12, symbol='circle-open', line=dict(width=2))
        ))

    # Dodajemo NR rezultate (zeleni križići)
    if nr_points:
        x_nr = np.array(nr_points)
        y_nr = np.array([f(xi) for xi in x_nr])
        fig.add_trace(go.Scatter(
            x=x_nr, y=y_nr,
            mode='markers',
            name='Newton-Raphson rezultati',
            marker=dict(color='green', size=12, symbol='x', line=dict(width=2))
        ))

    # Postavljamo layout
    fig.update_layout(
        title=title,
        xaxis_title='x',
        yaxis_title='f(x)',
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'),
        plot_bgcolor='white',
        hovermode='closest',
        showlegend=True,
        width=900,
        height=600
    )

    fig.show()

    # Ispis rezultata
    print(f"\n{'='*60}")
    print(f"Rezime za funkciju: {title}")
    print(f"{'='*60}")
    print(f"Početne tačke: {x0_values}")
    print(f"Pronađene tačke: {[f'{x:.6f}' for x in nr_points]}")
    if local_extrema:
        print(f"Pravi lokalni ekstremi: {[f'{x:.6f}' for x in local_extrema]}")
    print(f"{'='*60}\n")


# ============================================================================
# TEST FUNKCIJA 1: f(x) = 3x^2 - 1
# ============================================================================

"""
# 🔬 Test 1: $f_1(x) = 3x^2 - 1$

### Analiza:
- Ovo je kvadratna funkcija (parabola)
- Ima jedan ekstrem u x = 0 (minimum)
- f'(x) = 6x
- f''(x) = 6

### Testiranje sa početnim tačkama: x₀ ∈ {1, 10}
"""

print("\n" + "="*70)
print("TEST 1: f(x) = 3x² - 1")
print("="*70)

x0_list_1 = [1, 10]
nr_results_1 = []

for x0 in x0_list_1:
    print(f"\n--- Početna tačka x0 = {x0} ---")
    x_opt, history, iters = NR(f1, df1, ddf1, x0=x0, N=100, eps=1e-6)
    nr_results_1.append(x_opt)

# Crtamo grafik
local_extrema_1 = [0]  # Jedini ekstrem je u x=0
plot_function_with_results(
    f=f1,
    x_range=(-2, 12),
    nr_points=nr_results_1,
    local_extrema=local_extrema_1,
    title='$f_1(x) = 3x^2 - 1$',
    x0_values=x0_list_1
)


# ============================================================================
# TEST FUNKCIJA 2: f(x) = -(16x^2 - 24x + 5)e^(-x)
# ============================================================================

"""
# 🔬 Test 2: $f_2(x) = -(16x^2 - 24x + 5)e^{-x}$

### Analiza:
- Složenija funkcija sa eksponencijalnim članom
- Ima lokalne ekstreme koje trebamo pronaći
- Testiramo sa x₀ ∈ {0.5, 2}
"""

print("\n" + "="*70)
print("TEST 2: f(x) = -(16x² - 24x + 5)e^(-x)")
print("="*70)

x0_list_2 = [0.5, 2]
nr_results_2 = []

for x0 in x0_list_2:
    print(f"\n--- Početna tačka x0 = {x0} ---")
    x_opt, history, iters = NR(f2, df2, ddf2, x0=x0, N=100, eps=1e-6)
    nr_results_2.append(x_opt)

# Za lokalne ekstreme, možemo ih pronaći numerički ili analitički
# Rješavamo f'(x) = 0: -e^(-x)[-16x^2 + 56x - 29] = 0
# -16x^2 + 56x - 29 = 0
# x = (56 ± sqrt(56^2 - 4*16*29)) / (2*16)
discriminant = 56**2 - 4*16*29
x_ext1 = (56 + np.sqrt(discriminant)) / 32
x_ext2 = (56 - np.sqrt(discriminant)) / 32
local_extrema_2 = [x_ext2, x_ext1]

plot_function_with_results(
    f=f2,
    x_range=(-1, 5),
    nr_points=nr_results_2,
    local_extrema=local_extrema_2,
    title='$f_2(x) = -(16x^2 - 24x + 5)e^{-x}$',
    x0_values=x0_list_2
)


# ============================================================================
# TEST FUNKCIJA 3: f(x) = sin(x) + sin(10x/3)
# ============================================================================

"""
# 🔬 Test 3: $f_3(x) = \\sin(x) + \\sin(\\frac{10x}{3})$

### Analiza:
- Trigonometrijska funkcija sa periodicnim svojstvima
- Ima više lokalnih ekstrema
- Testiramo sa x₀ ∈ {3, 6, 7}
"""

print("\n" + "="*70)
print("TEST 3: f(x) = sin(x) + sin(10x/3)")
print("="*70)

x0_list_3 = [2, 3, 6, 7]
nr_results_3 = []

for x0 in x0_list_3:
    print(f"\n--- Početna tačka x0 = {x0} ---")
    x_opt, history, iters = NR(f3, df3, ddf3, x0=x0, N=100, eps=1e-6)
    nr_results_3.append(x_opt)

# Lokalne ekstreme pronalazimo numerički
# Tražimo gdje je df3(x) ≈ 0 u opsegu
x_test = np.linspace(0, 10, 10000)
df3_vals = df3(x_test)
# Pronalazimo gdje derivacija mijenja predznak
sign_changes = np.where(np.diff(np.sign(df3_vals)))[0]
local_extrema_3 = [x_test[i] for i in sign_changes]

plot_function_with_results(
    f=f3,
    x_range=(0, 10),
    nr_points=nr_results_3,
    local_extrema=local_extrema_3,
    title='$f_3(x) = \\sin(x) + \\sin(\\frac{10x}{3})$',
    x0_values=x0_list_3
)


# ============================================================================
# TEST FUNKCIJA 4: f(x) = e^(-x)
# ============================================================================

"""
# 🔬 Test 4: $f_4(x) = e^{-x}$

### Analiza:
- Eksponencijalna funkcija koja monotono opada
- f'(x) = -e^(-x) < 0 za svako x (monotono opadajuća)
- f''(x) = e^(-x) > 0 za svako x (konveksna)
- **NEMA lokalnih ekstrema!** Funkcija je striktno monotona
- Testiramo sa x₀ ∈ {1, 10}
"""

print("\n" + "="*70)
print("TEST 4: f(x) = e^(-x)")
print("="*70)

x0_list_4 = [1, 10]
nr_results_4 = []

for x0 in x0_list_4:
    print(f"\n--- Početna tačka x0 = {x0} ---")
    x_opt, history, iters = NR(f4, df4, ddf4, x0=x0, N=100, eps=1e-6)
    nr_results_4.append(x_opt)

# Nema lokalnih ekstrema - funkcija je monotona
local_extrema_4 = []

plot_function_with_results(
    f=f4,
    x_range=(-1, 12),
    nr_points=nr_results_4,
    local_extrema=local_extrema_4,
    title='$f_4(x) = e^{-x}$',
    x0_values=x0_list_4
)


# ============================================================================
# ZAKLJUČCI
# ============================================================================

"""
# 📝 Zaključci i analiza rezultata

## Opažanja po funkcijama:

### Funkcija 1: $f_1(x) = 3x^2 - 1$
- ✅ Algoritam uspješno pronalazi minimum u x = 0
- Konvergencija je brza za obje početne tačke
- Kvadratna funkcija je idealan slučaj za Newton-Raphson

### Funkcija 2: $f_2(x) = -(16x^2 - 24x + 5)e^{-x}$
- ✅ Algoritam pronalazi različite lokalne ekstreme zavisno od početne tačke
- Pokazuje važnost izbora početne tačke
- Oba rezultata su validni lokalni ekstremi

### Funkcija 3: $f_3(x) = \\sin(x) + \\sin(\\frac{10x}{3})$
- ✅ Funkcija ima mnogo lokalnih ekstrema
- Algoritam konvergira ka najbližem ekstremu od početne tačke
- Demonstrira ograničenje metode - nalazi samo lokalne, ne i globalne ekstreme

### Funkcija 4: $f_4(x) = e^{-x}$
- ⚠️ Funkcija NEMA lokalne ekstreme (striktno monotona)
- Algoritam ne može da konvergira jer tražimo ekstrem koji ne postoji
- Ovo ilustruje kada metoda nije primjenjiva

## Opšti zaključci:

1. **Brzina konvergencije**: Newton-Raphson metoda ima kvadratnu konvergenciju
   kod "dobrih" funkcija (glatke, konveksne/konkavne)

2. **Zavisnost od početne tačke**: Različite početne tačke mogu dovesti do
   različitih lokalnih ekstrema

3. **Ograničenja metode**:
   - Nalazi samo lokalne ekstreme, ne garantuje globalni ekstrem
   - Zahtijeva da f''(x) ≠ 0
   - Ne radi na monotonim funkcijama
   - Može divergirati ili oscilirati sa lošim početnim tačkama

4. **Primjenjivost**: Metoda je efikasna za funkcije sa jasno definisanim
   lokalnim ekstremima i kada imamo dobru procjenu početne tačke

## Preporuke:

- Za pronalaženje globalnog ekstrema, kombinovati sa metodama globalnog pretraživanja
- Koristiti više početnih tačaka i porediti rezultate
- Provjeravati prirodu ekstrema (minimum/maximum) preko f''(x)
- Dodati dodatne provjere za robusnost (npr. line search)
"""

print("\n" + "="*70)
print("KRAJ LABORATORIJSKE VJEŽBE")
print("="*70)
print("\n✅ Svi zadaci uspješno urađeni!")
print("📊 Grafici prikazani koristeći Plotly")
print("📝 Rezultati analizirani i dokumentovani")

✅ Sve biblioteke uspješno učitane!
✅ Sve funkcije i njihovi izvodi definirani!

TEST 1: f(x) = 3x² - 1

--- Početna tačka x0 = 1 ---
✓ Konvergencija postignuta nakon 2 iteracija
  Početna tačka: x0 = 1.000000
  Finalna tačka: x* = 0.000000
  Vrijednost funkcije: f(x*) = -1.000000

--- Početna tačka x0 = 10 ---
✓ Konvergencija postignuta nakon 2 iteracija
  Početna tačka: x0 = 10.000000
  Finalna tačka: x* = 0.000000
  Vrijednost funkcije: f(x*) = -1.000000



Rezime za funkciju: $f_1(x) = 3x^2 - 1$
Početne tačke: [1, 10]
Pronađene tačke: ['0.000000', '0.000000']
Pravi lokalni ekstremi: ['0.000000']


TEST 2: f(x) = -(16x² - 24x + 5)e^(-x)

--- Početna tačka x0 = 0.5 ---
✓ Konvergencija postignuta nakon 4 iteracija
  Početna tačka: x0 = 0.500000
  Finalna tačka: x* = 0.631966
  Vrijednost funkcije: f(x*) = 2.007695

--- Početna tačka x0 = 2 ---
✓ Konvergencija postignuta nakon 4 iteracija
  Početna tačka: x0 = 2.000000
  Finalna tačka: x* = 2.868034
  Vrijednost funkcije: f(x*) = -3.850451



Rezime za funkciju: $f_2(x) = -(16x^2 - 24x + 5)e^{-x}$
Početne tačke: [0.5, 2]
Pronađene tačke: ['0.631966', '2.868034']
Pravi lokalni ekstremi: ['0.631966', '2.868034']


TEST 3: f(x) = sin(x) + sin(10x/3)

--- Početna tačka x0 = 2 ---
✓ Konvergencija postignuta nakon 5 iteracija
  Početna tačka: x0 = 2.000000
  Finalna tačka: x* = 2.296091
  Vrijednost funkcije: f(x*) = 1.728302

--- Početna tačka x0 = 3 ---
✓ Konvergencija postignuta nakon 5 iteracija
  Početna tačka: x0 = 3.000000
  Finalna tačka: x* = 3.387252
  Vrijednost funkcije: f(x*) = -1.199921

--- Početna tačka x0 = 6 ---
✓ Konvergencija postignuta nakon 3 iteracija
  Početna tačka: x0 = 6.000000
  Finalna tačka: x* = 6.217309
  Vrijednost funkcije: f(x*) = 0.888315

--- Početna tačka x0 = 7 ---
✓ Konvergencija postignuta nakon 1 iteracija
  Početna tačka: x0 = 7.000000
  Finalna tačka: x* = 7.000149
  Vrijednost funkcije: f(x*) = -0.316996



Rezime za funkciju: $f_3(x) = \sin(x) + \sin(\frac{10x}{3})$
Početne tačke: [2, 3, 6, 7]
Pronađene tačke: ['2.296091', '3.387252', '6.217309', '7.000149']
Pravi lokalni ekstremi: ['0.548055', '1.398140', '2.295230', '3.386339', '4.196420', '5.145515', '6.216622', '6.999700', '7.997800', '9.037904', '9.810981']


TEST 4: f(x) = e^(-x)

--- Početna tačka x0 = 1 ---
⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji 27!
   Pomjeram se za random offset...
⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji 28!
   Pomjeram se za random offset...
⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji 29!
   Pomjeram se za random offset...
⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji 30!
   Pomjeram se za random offset...
⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji 31!
   Pomjeram se za random offset...
⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji 32!
   Pomjeram se za random offset...
⚠️ Upozorenje: Drugi izvod blizu nule na iteraciji 33!
   Pomjeram se za random offset...
⚠️ 


Rezime za funkciju: $f_4(x) = e^{-x}$
Početne tačke: [1, 10]
Pronađene tačke: ['27.990000', '28.000000']


KRAJ LABORATORIJSKE VJEŽBE

✅ Svi zadaci uspješno urađeni!
📊 Grafici prikazani koristeći Plotly
📝 Rezultati analizirani i dokumentovani
